In [1]:
%run load_data_jester500.ipynb

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
  0%|          | 0/500 [00:00<?, ?it/s]

(100, 176, 3)
plt.imshow(img)


  0%|          | 2/500 [00:00<00:27, 17.79it/s]

(3, 125, 57, 16)
500
import Doing other things finish!!


  0%|          | 1/500 [00:00<01:11,  6.97it/s]

...

(3, 125, 57, 16)
3500
import swiping up finish!!


  0%|          | 2/500 [00:00<00:28, 17.28it/s]

(3, 125, 57, 16)
4000
import thumb up finish!!


100%|██████████| 500/500 [00:23<00:00, 21.62it/s]


(3, 125, 57, 16)
4500
import Thumb Down finish!!
4500
[0 0 0 ... 8 8 8]
X_Train shape: (4500, 3, 125, 57, 16)
(4500, 3, 125, 57, 16) train samples


# Define Model

In [2]:
from keras.optimizers import Adam, rmsprop
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD
from keras.layers import Dense,Dropout,Conv3D,Input,MaxPool3D,Flatten,Activation,ZeroPadding3D,BatchNormalization
from keras.regularizers import l2
from keras.models import Model
import keras

input_shape = (3,img_rows,img_cols,img_depth) ##channel first ##channel,spatial_dim1, spatial_dim2, spatial_dim3,
weight_decay = 0.005
nb_classes = 9

## CNN Training parameters

In [3]:
nb_classes = 9
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
print(Y_train)
input_shape = (3, img_rows, img_cols, img_depth)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [5]:
model = Sequential()

model.add(Conv3D(64,(3,3,3),strides=(1,1,1), input_shape= input_shape, name='conv1', padding='same',data_format="channels_first", activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(MaxPool3D((2,2,2),strides=(2,2,2), name='pool1',padding='same',data_format="channels_first"))

model.add(Conv3D(64,(3,3,3),strides=(1,1,1), input_shape= input_shape, name='conv2', padding='same',data_format="channels_first", activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(MaxPool3D((2,2,2),strides=(2,2,2), name='pool2',padding='same',data_format="channels_first"))

#fully connected layer
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_regularizer=l2(weight_decay)))
model.add(Dropout(0.5))
model.add(Dense(nb_classes,activation='softmax',kernel_regularizer=l2(weight_decay)))

print(model.summary())
#opt = keras.optimizers.Adam(lr=0.001)
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt ,metrics=['accuracy'])

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling3d_4/MaxPool3D' (op: 'MaxPool3D') with input shapes: [?,128,27,15,1].

## split data

In [ ]:
X_train_new, X_val_new, y_train_new,y_val_new =  train_test_split(train_set, Y_train, test_size=0.2, random_state=4)
X_train_new = X_train_new.astype('float32')
X_val_new = X_val_new.astype('float32')
X_train_new /= 255.
X_val_new /= 255.

In [ ]:
batch_size = 30
nb_epoch = 500

# Use ModelCheckpoint to save model and weights
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'C3D_model_jester_RGB.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

# earlystop
earlystop = EarlyStopping(monitor='val_loss', patience=50, verbose=1)


hist = model.fit(X_train_new, y_train_new, 
                 validation_data=(X_val_new,y_val_new),
                 batch_size=batch_size,
                 epochs = nb_epoch, 
                 shuffle=True, 
                 callbacks=[checkpoint, earlystop])

In [ ]:
# loading our save model
print("Loading trained model")
model = load_model(model_path)

In [ ]:
score = model.evaluate(X_val_new, y_val_new, batch_size=batch_size)
print('Test score:', score[0])
print('Test accuracy:', score[1]) 

In [ ]:
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']

plt.figure(1,figsize=(7,5))
plt.plot(train_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.title("Learning Curve")
plt.legend(loc='best')
print (plt.style.available) # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(train_acc, label="training_acc")
plt.plot(val_acc, label="validation_acc")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.title("Learning Curve")
plt.legend(loc='best')
print (plt.style.available) # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])


In [ ]:
import itertools
def confusion_matrix_plot(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix

In [ ]:
#model.fit(X_train_new, y_train_new)
#y_pred = model.predict(X_val_new)
y_pred = model.predict(train_set)

#a = np.argmax(y_val_new, axis=1)
a = np.argmax(Y_train, axis=1)
b = np.argmax(y_pred, axis=1)
#print("benchmark : ",a)
#print("predict : ",b)

In [ ]:
met = confusion_matrix(a, b)
np.set_printoptions(precision=2)
print(met)

In [ ]:
confusion_matrix_plot(met, normalize=True, classes=['1', '2', '3', '4', '5', '6', '7', '8', '9'])